In [6]:
import datetime
import logging
import os
import random
import tkinter as tk
import configparser
import google.generativeai as genai
import mysql.connector
import pytz
from tkinter import ttk, messagebox

# Importa IPython para obter o diretório do notebook
from IPython import get_ipython


class StudyTracker:
    def __init__(self, root):
        self.root = root
        self.root.title("Classificador de Assuntos")

        # Configurações do MySQL
        self.config = configparser.ConfigParser()

        # Usando o caminho absoluto para o config.ini (como no SonoApp/RefeicaoApp)
        config_path = r"C:\Users\robso\PERFORMANCE_ESTUDOS2\Credenciais\config.ini"  # Caminho absoluto
        print(f"Caminho do arquivo de configuração: {config_path}")  # Para debug
        try:
            self.config.read(config_path)
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao ler o arquivo de configuração: {e}")
            root.destroy()  # Encerra o aplicativo se não conseguir ler o arquivo de configuração
            return

        self.db_config = dict(self.config['database'])

        raise_on_warnings_str = self.db_config.get('raise_on_warnings', 'True')
        if raise_on_warnings_str.lower() == 'true':
            raise_on_warnings = True
        elif raise_on_warnings_str.lower() == 'false':
            raise_on_warnings = False
        else:
            raise_on_warnings = True

        self.db_config['raise_on_warnings'] = raise_on_warnings

        # Configuração da API do Gemini
        self.gemini_api_key = self.load_gemini_api_key()
        if not self.gemini_api_key:
            messagebox.showerror("Erro", "API Key do Gemini não encontrada.")
            root.destroy()
            return

        genai.configure(api_key=self.gemini_api_key)

        # Carrega o prompt do arquivo
        self.prompt = self.load_prompt()
        if not self.prompt:
            messagebox.showerror("Erro", "Prompt do Gemini não encontrado.")
            root.destroy()
            return

        # Fuso horário de Brasília
        self.brasilia_tz = pytz.timezone('America/Sao_Paulo')

        # Componentes da Interface Gráfica
        self.descricao_label = ttk.Label(root, text="Descrição:")
        self.descricao_entry = ttk.Entry(root, width=50)
        self.fonte_label = ttk.Label(root, text="Fonte:")
        self.fonte_entry = ttk.Entry(root, width=50)
        self.classificar_button = ttk.Button(
            root, text="Classificar Assunto", command=self.classificar_assunto)
        self.resultado_label = ttk.Label(root, text="Resultado:")
        self.resultado_output = tk.Text(root, height=4, width=50)
        self.resultado_output.config(state=tk.DISABLED)

        self.iniciar_button = ttk.Button(
            root, text="Iniciar Estudo", command=self.iniciar_estudo)
        self.encerrar_button = ttk.Button(
            root, text="Encerrar Estudo", command=self.encerrar_estudo)
        self.gerar_avaliacao_button = ttk.Button(
            root, text="Gerar Avaliação", command=self.gerar_avaliacao)

        self.inicio_tempo_label = ttk.Label(root, text="Início:")
        self.inicio_tempo_output = ttk.Label(root, text="")

        self.termino_tempo_label = ttk.Label(root, text="Término:")
        self.termino_tempo_output = ttk.Label(root, text="")

        # Layout
        self.descricao_label.grid(row=0, column=0, padx=5, pady=5, sticky="w")
        self.descricao_entry.grid(row=0, column=1, padx=5, pady=5, sticky="ew")
        self.fonte_label.grid(row=1, column=0, padx=5, pady=5, sticky="w")
        self.fonte_entry.grid(row=1, column=1, padx=5, pady=5, sticky="ew")
        self.classificar_button.grid(
            row=2, column=0, columnspan=2, padx=5, pady=5)
        self.resultado_label.grid(row=3, column=0, padx=5, pady=5, sticky="w")
        self.resultado_output.grid(
            row=3, column=1, padx=5, pady=5, sticky="ew")

        self.iniciar_button.grid(row=4, column=0, padx=5, pady=5, sticky="e")
        self.inicio_tempo_output.grid(row=4, column=1, padx=5, pady=5, sticky="w")

        self.encerrar_button.grid(row=5, column=0, padx=5, pady=5, sticky="e")
        self.termino_tempo_output.grid(row=5, column=1, padx=5, pady=5, sticky="w")

        self.gerar_avaliacao_button.grid(
            row=6, column=0, columnspan=2, padx=5, pady=5)

        self.inicio_tempo_label.grid_remove()
        self.termino_tempo_label.grid_remove()

        # Variáveis de estado
        self.inicio_tempo = None
        self.termino_tempo = None
        self.id_assunto = None
        self.ultimo_id_estudo = None

        root.columnconfigure(1, weight=1)

    def load_gemini_api_key(self):
        api_key_path = r"C:\Users\robso\PERFORMANCE_ESTUDOS2\Credenciais\api_key.txt"  # Caminho absoluto
        try:
            with open(api_key_path, "r") as f:
                return f.read().strip()
        except FileNotFoundError:
            return None

    def load_prompt(self):
        prompt_path = r"C:\Users\robso\PERFORMANCE_ESTUDOS2\Prompts gemini\prompt_assunto_geral.txt"  # Caminho absoluto
        try:
            with open(prompt_path, "r") as f:
                return f.read()
        except FileNotFoundError:
            return None

    def load_questionario_prompt(self):
        """Carrega o prompt do questionário do arquivo."""
        prompt_path = r"C:\Users\robso\PERFORMANCE_ESTUDOS2\Prompts gemini\prompt_gerar_questionario.txt" # Caminho absoluto
        try:
            with open(prompt_path, "r", encoding="utf-8") as f:
                return f.read()
        except FileNotFoundError:
            return None

    def classificar_assunto(self):
        descricao = self.descricao_entry.get()
        if not descricao:
            messagebox.showerror("Erro", "Por favor, insira uma descrição.")
            return

        try:
            cnx = mysql.connector.connect(**self.db_config)
            cursor = cnx.cursor()

            query = "SELECT id, assunto FROM assuntos_gerais"
            cursor.execute(query)
            assuntos = cursor.fetchall()

            prompt = self.prompt.format(descricao=descricao)
            for id, assunto in assuntos:
                prompt += f"- {id}: {assunto}\n"
            prompt += "Responda apenas com o número do ID do assunto que melhor se encaixa."

            model = genai.GenerativeModel('gemini-pro',
                                            generation_config=genai.types.GenerationConfig(
                                                temperature=1.2  # Ajuste a temperatura aqui!
                                            ))
            response = model.generate_content(prompt)

            try:
                self.id_assunto = int(response.text.strip())
            except ValueError:
                messagebox.showerror(
                    "Erro",
                    "Não foi possível extrair o ID do assunto da resposta do Gemini.")
                return

            for id, assunto in assuntos:
                if id == self.id_assunto:
                    assunto_selecionado = assunto
                    break
            else:
                messagebox.showerror(
                    "Erro",
                    f"ID do assunto '{self.id_assunto}' não encontrado no banco de dados.")
                return

            self.resultado_output.config(state=tk.NORMAL)
            self.resultado_output.delete("1.0", tk.END)
            self.resultado_output.insert(
                tk.END, f"ID: {self.id_assunto}\nAssunto: {assunto_selecionado}")
            self.resultado_output.config(state=tk.DISABLED)

        except mysql.connector.Error as err:
            messagebox.showerror("Erro", f"Erro ao acessar o MySQL: {err}")
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao classificar o assunto: {e}")
        finally:
            if 'cnx' in locals() and cnx.is_connected():
                cursor.close()
                cnx.close()

    def iniciar_estudo(self):
        self.inicio_tempo = datetime.datetime.now(self.brasilia_tz)
        tempo_formatado = self.inicio_tempo.strftime("%Y-%m-%d %H:%M:%S")

        self.inicio_tempo_output.config(text=tempo_formatado)
        self.termino_tempo_output.config(text="")

    def encerrar_estudo(self):
        self.termino_tempo = datetime.datetime.now(self.brasilia_tz)
        tempo_formatado = self.termino_tempo.strftime("%Y-%m-%d %H:%M:%S")

        self.termino_tempo_output.config(text=tempo_formatado)

        self.salvar_estudo()

        self.limpar_campos()

    def salvar_estudo(self):
        if not self.inicio_tempo or not self.termino_tempo or not self.id_assunto:
            messagebox.showerror(
                "Erro",
                "Por favor, inicie, finalize o estudo e classifique o assunto antes de salvar.")
            return

        try:
            cnx = mysql.connector.connect(**self.db_config)
            cursor = cnx.cursor()

            descricao = self.descricao_entry.get().lower()
            fonte = self.fonte_entry.get().lower()

            inicio_formatado = self.inicio_tempo.strftime('%Y-%m-%d %H:%M:%S')
            termino_formatado = self.termino_tempo.strftime('%Y-%m-%d %H:%M:%S')

            query = """
            INSERT INTO habitos.estudos (inicio, termino, id_assunto, descricao, fonte)
            VALUES (%s, %s, %s, %s, %s)
            """
            values = (inicio_formatado, termino_formatado,
                      self.id_assunto, descricao, fonte)

            cursor.execute(query, values)
            cnx.commit()

            self.ultimo_id_estudo = cursor.lastrowid

            messagebox.showinfo("Sucesso", "Dados do estudo salvos com sucesso!")

        except mysql.connector.Error as err:
            messagebox.showerror("Erro", f"Erro ao salvar os dados: {err}")
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao salvar o estudo: {e}")

        finally:
            if 'cnx' in locals() and cnx.is_connected():
                cursor.close()
                cnx.close()

    def limpar_campos(self):
        self.descricao_entry.delete(0, tk.END)
        self.fonte_entry.delete(0, tk.END)
        self.resultado_output.config(state=tk.NORMAL)
        self.resultado_output.delete("1.0", tk.END)
        self.resultado_output.config(state=tk.DISABLED)
        self.inicio_tempo_output.config(text="")
        self.termino_tempo_output.config(text="")
        self.inicio_tempo = None
        self.termino_tempo = None
        self.id_assunto = None
        self.ultimo_id_estudo = None

    def gerar_avaliacao(self):
        try:
            cnx = mysql.connector.connect(**self.db_config)
            cursor = cnx.cursor()

            hoje = datetime.datetime.now(self.brasilia_tz).date()

            query = """
            SELECT id, descricao, fonte
            FROM habitos.estudos
            WHERE DATE(inicio) = %s
            """
            cursor.execute(query, (hoje,))
            estudos_do_dia = cursor.fetchall()

            if not estudos_do_dia:
                messagebox.showinfo(
                    "Aviso", "Nenhum estudo registrado para o dia de hoje.")
                return

            lista_estudos = []
            for id_estudo, descricao, fonte in estudos_do_dia:
                lista_estudos.append({
                    "id": id_estudo,
                    "descricao": descricao,
                    "fonte": fonte
                })

            self.gerar_questoes_em_lote(lista_estudos)

        except mysql.connector.Error as err:
            messagebox.showerror("Erro", f"Erro ao acessar o MySQL: {err}")
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao gerar a avaliação: {e}")
        finally:
            if 'cnx' in locals() and cnx.is_connected():
                cursor.close()
                cnx.close()

    def gerar_questoes_em_lote(self, lista_estudos):
        # Carrega o prompt do arquivo
        prompt_base = self.load_questionario_prompt()
        if not prompt_base:
            messagebox.showerror("Erro", "Prompt do questionário não encontrado.")
            return

        num_estudos = len(lista_estudos)
        questoes_por_estudo = 10 // num_estudos  # Divisão inteira
        resto_questoes = 10 % num_estudos  # Questões restantes

        todas_questoes = []
        for i, estudo in enumerate(lista_estudos):
            # Calcula o número de questões para este estudo
            num_questoes = questoes_por_estudo + (1 if i < resto_questoes else 0)

            # Formata o material de estudo para este estudo específico
            material_estudo = f"""
            Descrição: {estudo["descricao"]}, Fonte: {estudo["fonte"]}
            """
            # Formata o prompt com o material de estudo específico
            prompt_avaliacao = prompt_base.format(estudos=material_estudo)

            # Adiciona a instrução para gerar um número limitado de questões
            prompt_avaliacao += f"""
            Gere exatamente {num_questoes} questões com base no material fornecido.
            """

            # Tentativa de gerar questões para este estudo até obter o número desejado
            tentativas = 0
            while tentativas < 5:  # Limita o número de tentativas
                model = genai.GenerativeModel('gemini-pro',
                                               generation_config=genai.types.GenerationConfig(
                                                   temperature=1.2  # Ajuste a temperatura aqui!
                                               ))
                response = model.generate_content(prompt_avaliacao)
                questoes = self.processar_questoes(response.text, [estudo])  # Passa apenas este estudo

                # Verifica se o número de questões geradas é o esperado
                if len(questoes) == num_questoes:
                    todas_questoes.extend(questoes)
                    break  # Sai do loop de tentativas para este estudo
                else:
                    tentativas += 1
                    print(f"Geradas {len(questoes)} questões para o estudo {estudo['id']}, esperado {num_questoes}. Tentando novamente ({tentativas}/5)...")

        # Verifica se o número total de questões é 10
        if len(todas_questoes) == 10:
            self.questoes = todas_questoes
            self.iniciar_questionario(self.questoes)
        else:
            messagebox.showinfo("Aviso", f"Não foi possível gerar exatamente 10 questões válidas após várias tentativas. Geradas {len(todas_questoes)} questões.")

    def processar_questoes(self, texto_response, lista_estudos):
        questoes = texto_response.strip().split("Pergunta:")
        questoes = [q.strip() for q in questoes if q.strip()]

        questoes_formatadas = []
        for i, questao in enumerate(questoes):
            linhas = questao.split("\n")
            linhas = [linha.strip() for linha in linhas if linha.strip()]

            if len(linhas) < 7:
                print(f"Questão incompleta encontrada. Ignorando: {questao}")
                continue

            pergunta = linhas[0].strip()

            # Verificação da alternativa "Todas as alternativas acima"
            alternativas = []
            for j in range(1, 6):
                alternativas.append(linhas[j][3:].strip())

            resposta_correta_linha = linhas[6]
            if "Resposta:" not in resposta_correta_linha:
                print(f"Linha de resposta incorreta. Ignorando: {questao}")
                continue

            resposta_correta = resposta_correta_linha.split("Resposta:")[1].strip()

            alternativas_invalidas = [
                "todas as alternativas acima",
                "todas as opções acima",
                "todas as alternativas corretas",
                "nenhuma das alternativas acima",
                "nenhuma das opções acima",
                "ambas a e b",
                "a e b",
                "a e c",
                "b e c"
            ]

            questao_invalida = False
            for alternativa in alternativas:
                if any(frase in alternativa.lower() for frase in alternativas_invalidas):
                    questao_invalida = True
                    break

            if questao_invalida:
                print(f"Questão com alternativa inválida encontrada. Descartando: {questao}")
                continue

            indice_resposta = ord(resposta_correta.upper()) - ord('A')

            if indice_resposta < 0 or indice_resposta >= len(alternativas):
                print(f"Índice de resposta incorreto. Ignorando: {questao}")
                continue

            resposta_correta_texto = alternativas[indice_resposta]

            # Atribui o ID do estudo à questão
            id_estudo = lista_estudos[0]["id"]  # Usa o ID do estudo atual

            questoes_formatadas.append({
                "pergunta": pergunta,
                "alternativas": alternativas,
                "resposta_correta": resposta_correta_texto,
                "id_estudo": id_estudo
            })

        return questoes_formatadas

    def iniciar_questionario(self, questoes):
        # Obtenha as coordenadas da janela principal
        x = self.root.winfo_x()
        y = self.root.winfo_y()
        width = self.root.winfo_width()

        # Calcule a posição para a nova janela aparecer ao lado
        new_x = x + width + 10  # Adiciona uma pequena distância de 10 pixels

        self.questionario_window = tk.Toplevel(self.root)
        self.questionario_window.title("Avaliação")
        self.questionario_window.geometry(f"800x300+{new_x}+{y}")  # Define a posição

        self.questoes = questoes
        self.questao_atual_index = 0
        self.respostas = []  # Lista para armazenar as respostas
        self.inicio_tempo_questao = None  # Guarda o tempo de início da questão atual

        self.exibir_proxima_questao()

    def exibir_proxima_questao(self):
        if self.questao_atual_index >= len(self.questoes):
            self.salvar_respostas_no_banco()  # Salva as respostas no final
            messagebox.showinfo("Avaliação Concluída",
                                "Você respondeu a todas as questões.")
            self.questionario_window.destroy()
            return

        questao = self.questoes[self.questao_atual_index]

        for widget in self.questionario_window.winfo_children():
            widget.destroy()

        # Guarda o tempo de início da questão
        self.inicio_tempo_questao = datetime.datetime.now(self.brasilia_tz)

        # Inclui o ID do estudo na criação do questionário
        self.questionario = QuestionarioUnica(
            self.questionario_window,
            questao["pergunta"],
            questao["alternativas"],
            questao["resposta_correta"],
            self.proxima_questao,
            self.questao_atual_index + 1,
            questao["id_estudo"],
            self
        )

    def proxima_questao(self, resposta_selecionada, acertou):
        # Registra o tempo de término e a resposta
        termino_tempo_questao = datetime.datetime.now(self.brasilia_tz)

        # Adiciona a resposta à lista
        self.respostas.append({
            "id_estudo": self.questoes[self.questao_atual_index]["id_estudo"],
            "pergunta": self.questao_atual_index + 1,
            "inicio": self.inicio_tempo_questao,
            "termino": termino_tempo_questao,
            "acerto": 1 if acertou else 0,
            "resposta_selecionada": resposta_selecionada
        })

        self.questao_atual_index += 1
        self.exibir_proxima_questao()

    def salvar_respostas_no_banco(self):
        try:
            cnx = mysql.connector.connect(**self.db_config)
            cursor = cnx.cursor()

            query = """
            INSERT INTO habitos.notas (id_estudo, pergunta, inicio, termino, acerto)
            VALUES (%s, %s, %s, %s, %s)
            """

            for resposta in self.respostas:
                values = (
                    resposta["id_estudo"],
                    resposta["pergunta"],
                    resposta["inicio"].strftime('%Y-%m-%d %H:%M:%S'),
                    resposta["termino"].strftime('%Y-%m-%d %H:%M:%S'),
                    resposta["acerto"]
                )
                cursor.execute(query, values)

            cnx.commit()
            messagebox.showinfo("Sucesso", "Respostas salvas no banco de dados!")

        except mysql.connector.Error as err:
            messagebox.showerror("Erro", f"Erro ao salvar as respostas: {err}")
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao salvar as respostas: {e}")
        finally:
            if 'cnx' in locals() and cnx.is_connected():
                cursor.close()
                cnx.close()


class QuestionarioUnica:
    def __init__(self, master, pergunta, alternativas, resposta_correta, proxima_questao_callback, questao_numero, id_estudo, study_tracker):
        self.master = master
        self.questao_numero = questao_numero
        self.id_estudo = id_estudo  # Armazena o ID do estudo
        self.resposta_correta = resposta_correta
        self.study_tracker = study_tracker
        master.title(f"Questão {self.questao_numero} - id {self.id_estudo}")  # Inclui o número e ID no título

        self.pergunta = pergunta
        self.alternativas = alternativas
        self.resposta_selecionada = tk.StringVar()
        self.resposta_selecionada.set(None)
        self.proxima_questao_callback = proxima_questao_callback

        self.label_pergunta = tk.Label(
            master, text=self.pergunta, wraplength=700)
        self.label_pergunta.pack(pady=10)

        self.radio_buttons = []
        for i, alternativa in enumerate(self.alternativas):
            radio_button = tk.Radiobutton(master, text=alternativa, variable=self.resposta_selecionada,
                                          value=alternativa)
            radio_button.pack(anchor=tk.W, padx=20)
            self.radio_buttons.append(radio_button)

        self.botao_enviar = tk.Button(
            master, text="Enviar Resposta", command=self.verificar_resposta)
        self.botao_enviar.pack(pady=20)

    def verificar_resposta(self):
        if not self.resposta_selecionada.get():
            messagebox.showinfo("Atenção", "Por favor, selecione uma alternativa.")
            return

        resposta_usuario = self.resposta_selecionada.get()
        acertou = resposta_usuario == self.resposta_correta

        if acertou:
            messagebox.showinfo("Resultado", "Resposta Correta!")
        else:
            messagebox.showinfo("Resultado",
                                f"Resposta Incorreta. A resposta correta é: {self.resposta_correta}")

        # Chama o callback do StudyTracker
        self.study_tracker.proxima_questao(resposta_usuario, acertou)


if __name__ == "__main__":
    root = tk.Tk()
    style = ttk.Style(root)
    style.theme_use('clam')
    app = StudyTracker(root)
    root.mainloop()

Caminho do arquivo de configuração: C:\Users\robso\PERFORMANCE_ESTUDOS2\Credenciais\config.ini
